In [ ]:
!pip install gdown 

In [5]:
!gdown --id 1ICytt-R6xV5ZLwHGCEvpITSaE_eejmUu --output Amazon.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ICytt-R6xV5ZLwHGCEvpITSaE_eejmUu
To: /content/Amazon.zip
100% 314M/314M [00:03<00:00, 86.3MB/s]


In [6]:
!unzip /content/Amazon.zip

Archive:  /content/Amazon.zip
   creating: Amazon-book-KG/
  inflating: Amazon-book-KG/hop3.kg  
  inflating: Amazon-book-KG/relation.kg  
  inflating: Amazon-book-KG/link.kg  
  inflating: Amazon-book-KG/hop1.kg  
  inflating: Amazon-book-KG/relation_full.kg  
  inflating: Amazon-book-KG/hop2.kg  
  inflating: Amazon-book-KG/extra.kg  


In [ ]:
import os
os.chdir('/content')

In [ ]:
import pandas as pd
users = []
with open('/content/Amazon-book-KG/link.kg', encoding='latin1') as f:
    for l in f:
        # print(l)
        id_2, id_1= l.strip().split('	')
        users.append({
            'id_2': (id_2),
            'id_1': (id_1)            })
users = pd.DataFrame(users).astype('category')

In [ ]:
ratings = pd.read_csv('/content/amazon_dataset_NEW.csv', index_col = 0)

In [ ]:
ratings['0000000116'].nunique()

8761

In [ ]:
import numpy as np
len(np.intersect1d(np.array(users['id_2'].unique()), np.array(ratings['0000000116'].unique())))

8761

In [ ]:
filenames = ['/content/Amazon-book-KG/hop1.kg', '/content/Amazon-book-KG/hop2.kg']
with open('/content/kg.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
relations = []
filenames = ['/content/Amazon-book-KG/hop1.kg', '/content/Amazon-book-KG/hop2.kg']
with open('/content/kg.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
              array = line.strip().split('\t')
              if array[1] not in relations:
                relations.append(array[1])

In [ ]:
import pandas as pd
ratings = pd.read_csv('/content/KGCN/data/movie/amazon_dataset_NEW.csv', index_col = 0)

In [ ]:
import os
os.chdir('/content/KGCN/data/movie')

In [4]:
!gdown --id 1x4Lhv-mw3xxiBpBYfxCDvmvFOfELCBKX --output kg.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1x4Lhv-mw3xxiBpBYfxCDvmvFOfELCBKX
To: /content/kg.txt
100% 1.63G/1.63G [00:08<00:00, 202MB/s]


In [ ]:
!gdown --id 1ufv-iXTF6i7uqmf-Vj0axJsr9RqzO-CH --output kg_final.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ufv-iXTF6i7uqmf-Vj0axJsr9RqzO-CH
To: /content/KGCN/data/movie/kg_final.txt
100% 31.6M/31.6M [00:00<00:00, 120MB/s] 


In [ ]:
!python -m pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 136 kB 5.1 MB/s 


In [ ]:
!pip install partd

Preprocessing

In [ ]:
import numpy as np
import dask.dataframe as dd
import pandas as pd
import pickle
import scipy.sparse as ssp

In [ ]:
def train_test_split_by_time(df, timestamp, user):
    df['train_mask'] = np.ones((len(df),), dtype=np.bool)
    df['val_mask'] = np.zeros((len(df),), dtype=np.bool)
    df['test_mask'] = np.zeros((len(df),), dtype=np.bool)
    df = dd.from_pandas(df, npartitions=10)
    def train_test_split(df):
        df = df.sort_values([timestamp])
        ##
        n_inter_items = df.shape[0]
        n_train_items = int(n_inter_items * 0.7)
        n_test_items = int(n_inter_items * 0.2)
        ##
        if df.shape[0] > 1:
            # df.iloc[:int(n_inter_items * 0.5), -3] = False
            df.iloc[-n_test_items:, -3] = False
            df.iloc[-n_test_items:, -1] = True
        if df.shape[0] > 2:
            df.iloc[n_train_items:-n_test_items, -3] = False
            df.iloc[n_train_items:-n_test_items, -2] = True
        return df
    df = df.groupby(user, group_keys=False).apply(train_test_split).compute(scheduler='processes').sort_index()
    # print(df[df[user] == df[user].unique()[0]].sort_values(timestamp))
    return df['train_mask'].to_numpy().nonzero()[0], \
           df['val_mask'].to_numpy().nonzero()[0], \
           df['test_mask'].to_numpy().nonzero()[0]

In [ ]:
import pickle
with open('/content/KGCN/data/movie/amazon_mapping_for_all.pkl_', 'rb') as f:
    mapping = pickle.load(f)

In [ ]:
ratings['AH2L9G3DQHHAJ'].nunique()

9968

In [ ]:
user_map = dict( enumerate(mapping['user'].cat.categories ) )
user_map = {value:key for (key,value) in user_map.items()}
ratings = ratings.replace({"AH2L9G3DQHHAJ": user_map})

In [ ]:
train_indices, val_indices, test_indices = train_test_split_by_time(ratings, '1019865600', 'AH2L9G3DQHHAJ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_

In [ ]:
df_train  = ratings.iloc[train_indices, :].copy()
df_test  = ratings.iloc[test_indices, :].copy()
df_val  = ratings.iloc[val_indices, :].copy()

In [ ]:
df_train.sort_values(by = ['AH2L9G3DQHHAJ'], inplace = True)
df_test.sort_values(by = ['AH2L9G3DQHHAJ'], inplace = True)
df_val.sort_values(by = ['AH2L9G3DQHHAJ'], inplace = True)

In [ ]:
ratings.sort_values(by = ['AH2L9G3DQHHAJ'], inplace = True)

In [ ]:
df_train.iloc[:, :4].to_csv('df_train.csv', index = False)
df_test.iloc[:, :4].to_csv('df_test.csv', index = False)
df_val.iloc[:, :4].to_csv('df_val.csv', index = False)

In [ ]:
ratings.iloc[:, :4].to_csv('ratings.csv', index = False)

DOUBLE CHECK

In [ ]:
# list(df_train_test[df_train_test['AH2L9G3DQHHAJ']==23]['0000000116'])

[1, 519, 527, 705, 938, 1238, 1421, 1839, 2624, 4714]

In [ ]:
df_train['AH2L9G3DQHHAJ'].nunique(), df_test['AH2L9G3DQHHAJ'].nunique(), df_val['AH2L9G3DQHHAJ'].nunique()

(9968, 9968, 9968)

Train test split

In [ ]:
os.chdir('/content')

In [1]:
!git clone https://github.com/hwwang55/KGCN.git

Cloning into 'KGCN'...
remote: Enumerating objects: 271, done.
remote: Total 271 (delta 0), reused 0 (delta 0), pack-reused 271
Receiving objects: 100% (271/271), 175.19 MiB | 38.77 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [2]:
!pip install gdown 

In [3]:
os.chdir('/content/KGCN/data/movie')

NameError: ignored

In [7]:
import os
os.chdir('/content/KGCN/src')

In [ ]:
df_train['AH2L9G3DQHHAJ'].nunique(), ratings['AH2L9G3DQHHAJ'].nunique()

(9968, 9968)

In [12]:
import pickle
with open('/content/KGCN/src/neg_samples_NEW.pkl', 'rb') as f:
    cheat = pickle.load(f)

In [40]:
import pickle
with open('/content/KGCN/src/neg_samples_NEW_2.pkl', 'rb') as f:
    cheat2 = pickle.load(f)

In [16]:
import numpy as np

In [17]:
cheat[2577][np.where(cheat[2577]!=7000)]

array([ 461,  558,  878,  924, 1023, 1100, 1685, 2218, 2405, 3144, 3202,
       3549, 4434, 5065, 5566, 5744, 7090, 7937, 8602])

In [41]:
cheat2[2577][np.where(cheat2[2577]!=7000)]

array([ 461,  878,  924, 1100, 2218, 2405, 3202, 3549, 5065, 5566, 7090,
       7937, 8602])

In [68]:
!python preprocess.py -d movie

reading item index to entity id file: ../data/movie/item_index2entity_id.txt ...
cnt 8761
reading rating file ...
converting rating file ...
number of users: 9968
number of items: 8761
user_index_old2new 23 2578 5678
converting kg file ...
number of entities (containing items): 697791
number of relations: 4
done


In [9]:
!pip install pip install tensorflow==1.15.5

     |████████████████████████████████| 110.5 MB 1.4 kB/s 
     |████████████████████████████████| 2.9 MB 32.2 MB/s 
     |████████████████████████████████| 3.8 MB 47.2 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 503 kB 59.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=d626bb665bba673ff84e5ad7579335bfc59fd0bb8990bd770ff0f4dc4f09e583
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tens

In [79]:
!python main.py

reading rating file ...
reading KG file ...
constructing knowledge graph ...
constructing adjacency matrix ...
data loaded.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2022-05-15 20:20:35.532008: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-15 20:20:35.535645: I tensorflow/core/platform/profil

In [80]:
import pickle
with open('/content/KGCN/src/KGCN_preds_for_amazon.pkl', 'rb') as f:
    res = pickle.load(f)

In [81]:
import numpy as np
res = np.array(res)

In [82]:
import numpy as np
import torch
import pickle



def read_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        lines = f.read().strip().split('\n')
    for line in lines:
        items = line.split(' ')[1:]
        items = [int(item) for item in items]
        # if items:
        #     self.n_items = max(self.n_items, max(items) + 1)
        data.append(items)
    return data

def calculate_metrics(eval_data, rec_items, topks):
    results = {'Precision': {}, 'Recall': {}, 'NDCG': {}}
    hit_matrix = np.zeros_like(rec_items, dtype=np.float32)
    for user in range(rec_items.shape[0]):
        for item_idx in range(rec_items.shape[1]):
            if rec_items[user, item_idx] in eval_data[user]:
                hit_matrix[user, item_idx] = 1.
    eval_data_len = np.array([len(items) for items in eval_data], dtype=np.int32)

    for k in topks:
        hit_num = np.sum(hit_matrix[:, :k], axis=1)
        precisions = hit_num / k
        with np.errstate(invalid='ignore'):
            recalls = hit_num / eval_data_len

        max_hit_num = np.minimum(eval_data_len, k)
        max_hit_matrix = np.zeros_like(hit_matrix[:, :k], dtype=np.float32)
        for user, num in enumerate(max_hit_num):
            max_hit_matrix[user, :num] = 1.
        denominator = np.log2(np.arange(2, k + 2, dtype=np.float32))[None, :]
        dcgs = np.sum(hit_matrix[:, :k] / denominator, axis=1)
        idcgs = np.sum(max_hit_matrix / denominator, axis=1)
        with np.errstate(invalid='ignore'):
            ndcgs = dcgs / idcgs

        user_masks = (max_hit_num > 0)
        results['Precision'][k] = precisions[user_masks].mean()
        results['Recall'][k] = recalls[user_masks].mean()
        results['NDCG'][k] = ndcgs[user_masks].mean()
    return results


def eval(recommendations, groud_truth_path, top_k):
        eval_data = read_data(groud_truth_path)
        metrics = calculate_metrics(eval_data, recommendations, top_k)
        precison = ''
        recall = ''
        ndcg = ''
        for k in top_k:
            precison += '{:.3f}%@{:d}, '.format(metrics['Precision'][k] * 100., k)
            recall += '{:.3f}%@{:d}, '.format(metrics['Recall'][k] * 100., k)
            ndcg += '{:.3f}%@{:d}, '.format(metrics['NDCG'][k] * 100., k)
        results = 'Precision: {:s}Recall: {:s}NDCG: {:s}'.format(precison, recall, ndcg)
        return results, metrics






In [83]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 0.662%@1, 0.674%@5, 0.641%@10, 0.621%@15, 0.564%@20, Recall: 0.150%@1, 0.964%@5, 1.823%@10, 2.742%@15, 3.326%@20, NDCG: 0.662%@1, 0.899%@5, 1.232%@10, 1.553%@15, 1.742%@20, ',
 {'NDCG': {1: 0.006621188,
   5: 0.008991225,
   10: 0.012316617,
   15: 0.015529677,
   20: 0.017419264},
  'Precision': {1: 0.006621188,
   5: 0.0067415736,
   10: 0.006410514,
   15: 0.0062132156,
   20: 0.0056430576},
  'Recall': {1: 0.0015041538024487488,
   5: 0.009644298665691638,
   10: 0.018228459293570663,
   15: 0.02741802579458174,
   20: 0.033255733175525264}})

In [ ]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 1.124%@1, 0.648%@5, 0.612%@10, 0.564%@15, 0.532%@20, Recall: 0.321%@1, 0.949%@5, 1.718%@10, 2.469%@15, 3.067%@20, NDCG: 1.124%@1, 0.967%@5, 1.281%@10, 1.536%@15, 1.733%@20, ',
 {'NDCG': {1: 0.011235955,
   5: 0.009667653,
   10: 0.0128057655,
   15: 0.015356598,
   20: 0.017332917},
  'Precision': {1: 0.011235955,
   5: 0.006480739,
   10: 0.006119583,
   15: 0.005638042,
   20: 0.0053170146},
  'Recall': {1: 0.0032078737863565103,
   5: 0.00948639835226836,
   10: 0.017180987631720268,
   15: 0.024689941551807457,
   20: 0.030673238592715365}})

In [ ]:
import numpy as np
kg_np = np.loadtxt('/content/KGCN/data/movie/kg_final' + '.txt', dtype=np.int64)


n_entity = len(set(kg_np[:, 0]) | set(kg_np[:, 2]))

In [ ]:
cnt = 0
for i in range(0, 186954):
  if i not in kg_np:
    cnt+=1

In [ ]:
cnt

0

In [ ]:
n_entity

186954

In [ ]:
def construct_kg(kg_np):
    print('constructing knowledge graph ...')
    kg = dict()
    for triple in kg_np:
        head = triple[0]
        relation = triple[1]
        tail = triple[2]
        # treat the KG as an undirected graph
        if head not in kg:
            kg[head] = []
        kg[head].append((tail, relation))
        if tail not in kg:
            kg[tail] = []
        kg[tail].append((head, relation))
    # print(kg[0], kg[1], kg[2])
    return kg

In [ ]:
kg = construct_kg(kg_np)

constructing knowledge graph ...


In [ ]:
kg[59]

[(57, 3), (57, 0)]

In [ ]:
for i in range(0, 187000):
  if i not in (kg.keys()):
    print(i)

186954
186955
186956
186957
186958
186959
186960
186961
186962
186963
186964
186965
186966
186967
186968
186969
186970
186971
186972
186973
186974
186975
186976
186977
186978
186979
186980
186981
186982
186983
186984
186985
186986
186987
186988
186989
186990
186991
186992
186993
186994
186995
186996
186997
186998
186999
